In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [ ]:
train.fillna(train.mean(), inplace=True)

In [ ]:
test.fillna(test.mean(), inplace=True)

In [ ]:
std_scale = StandardScaler().fit(train[train_numerical_cols])
train[train_numerical_cols] = std_scale.transform(train[train_numerical_cols])

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
inputcols.remove('readmission_status')
outputcols = allcols[3]
print(outputcols)

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
label = LabelEncoder()
for c in  X.columns:
    if(X[c].dtype == 'object'):
        X[c]=label.fit_transform(X[c])
    else:
        X[c]=X[c]

In [44]:
X.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,0.347324,-7.951567e-01,-0.474441,3,2,0.994578,5,2,-1.818248,0,2,-0.118219,-4.110915e-01,-1.386008e+00,-0.849116,-0.122062,-0.501727,-0.170324,-5.706394e-01,2.816280e-01,7.573758e-01,-8.050604e-14,-0.493791,4.147661e-01,-0.098641,0.003382,9.011102e-02,0.595402,-9.187153e-01,-0.423752,-1.148601,-2.407353e-15,-2.407353e-15,2.139204e-15,1.905845e-14,0.679073,-0.841450,3.542215,1.610465e-15,-0.697755,3.260997e-01,-2.964296e+00,-2.253897e+00,-1.035966,-0.989161,-1.047224,-0.998267,0.727253,0.098124,-1.935218e+00,-2.418484e+00,-0.753074,-1.205040,-0.759271,-1.216220,0.479167,-0.563143,0.423340,-1.643545,-1.964150,-2.344888e+00,-0.674415,-1.157987,-0.672017,-1.165366,3.821794,1.266390,2.042419e-15,0.000000,-0.407628,0.010092,-4.403744e-01,-1.718178e-02,1.245787,1.218709,-2.150716e-15,1.723599e-15,-0.280549,0.300422,-2.867673e-01,0.289759,4.589908e-01,0.133041,0.622974,-3.271545,0.000000,-1.187104e-15,-0.083209,-0.052430,-0.077393,-0.060685,4.239867e+00,1.315245e+00,-1.465665e+00,-1.311405e+00,-5.375520e-01,-5.139694e-01,0.275924,3.524088e-01,1.

In [45]:
print(len(inputcols))

181


In [48]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(181,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
	keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [49]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=200, batch_size=5)

Train on 91713 samples
Epoch 1/200
91713/91713 [==============================] - 61s 664us/sample - loss: -67622.7687 - accuracy: 0.0000e+00
Epoch 2/200
54595/91713 [================>.............] - ETA: 23s - loss: -479551.1502 - accuracy: 0.0000e+00

In [ ]:
print(model.predict(a))

In [ ]:
model = Sequential()
model.add(Dense(181, input_dim=90, activation='relu'))
model.add(Dense(181, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X, y, epochs=200, verbose=0)

In [ ]:
# Baseline model for the neural network. We choose a hidden layer of 10 neurons. The lesser number of neurons helps to eliminate the redundancies in the data and select the more important features.
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=181, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Evaluate model using standardized dataset. 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
pipeline.getModel()

In [ ]:
print('completed')